# Chapter 2 - Routing
For this example, we will use LangGraph to build a simple agent that routes user queries to one of two different "tools" (represented by simple functions) based on the user's input. This demonstrates LLM-based routing within a stateful workflow.

In [1]:
import os
from typing import Literal # Used for type hinting specific string values
from langchain_openai import ChatOpenAI # Or import your preferred model like ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser # To parse LLM output to string
# from langchain_core.pydantic_v1 import BaseModel, Field # For defining the graph state
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END # Core LangGraph components

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify if API key is present
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("OpenAI API key not found. Please check your .env file.")

# Initialize the OpenAI client and make the API call
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model="gpt-4o-mini", temperature=0.7,
    messages=[
        {"role": "user", "content": "Hello OpenAI!, Write a random small 1 line Thought of the day!"}
    ]
)
# Print the response
print(response.choices[0].message.content)

"Embrace the uncertainty of today; it may lead to the magic of tomorrow."


## --- Initialize the language model ---

In [3]:
# Use the appropriate class and model name for your provider (e.g., ChatGoogleGenerativeAI(model="gemini-pro"))
# Setting temperature to control creativity (0.7 is a common balance)
try:
    # Example for OpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    # Example for Google (uncomment and replace if using Google)
    # from langchain_google_genai import ChatGoogleGenerativeAI
    # llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    print(f"Language model initialized: {llm.model_name}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    print("Please ensure your API key is set correctly and the model name is valid.")
    llm = None # Set llm to None if initialization fails

Language model initialized: gpt-4o-mini


## --- Define the Graph State ---

In [4]:
# The state object that will be passed between nodes in the graph.
# It holds all the information needed for routing and processing throughout the workflow.
class GraphState(BaseModel):
   """Represents the state of our graph."""
   # The user's original question that initiates the workflow
   question: str = Field(description="The user's original question.")
   # The chosen route based on the analysis of the question. Literal ensures type safety.
   route: Literal["tool_a", "tool_b", "unclear"] | None = Field(
       description="The chosen route based on question analysis.",
       default=None # Default value is None before routing decision is made
   )
   # Placeholder for output from Tool A (if executed)
   tool_a_output: str | None = Field(
       description="Output from Tool A.",
       default=None
   )
   # Placeholder for output from Tool B (if executed)
   tool_b_output: str | None = Field(
       description="Output from Tool B.",
       default=None
   )
   # The final answer generated for the user
   final_answer: str | None = Field(
       description="The final answer to the user's question.",
       default=None
   )

## --- Define the Nodes (Steps in the Workflow) ---

In [5]:
# Node 1: Route the question
# This node's responsibility is to analyze the input question and decide the next path.
def route_question(state: GraphState) -> GraphState:
   """
   Analyzes the question and decides which tool to route to.
   Uses an LLM call for routing logic.
   """
   print("\n---ROUTE QUESTION---") # Print statement to show node execution
   question = state.question


   # Prompt the LLM to decide the route
   # The system prompt clearly defines the task and expected output format.
   route_prompt = ChatPromptTemplate.from_messages([
       ("system", """Analyze the user's question and determine if it is related to 'Tool A' or 'Tool B'.
        Tool A is related to processing data or performing calculations.
        Tool B is related to retrieving information or providing summaries.
        Output 'tool_a' if the question is primarily about Tool A.
        Output 'tool_b' if the question is primarily about Tool B.
        Output 'unclear' if the question is not clearly related to either, is ambiguous, or asks for something else.
        ONLY output one of the following words: tool_a, tool_b, unclear"""),
       ("user", "{question}") # The user's question is the input to this prompt
   ])


   # Create a chain for routing: Prompt -> LLM -> Parse Output
   route_chain = route_prompt | llm | StrOutputParser()


   # Invoke the routing chain to get the decision
   try:
       route_decision = route_chain.invoke({"question": question})
       print(f"LLM Routing decision raw output: '{route_decision}'")
       # Update the state with the routing decision after cleaning up whitespace and converting to lowercase
       state.route = route_decision.strip().lower()
       print(f"State updated with route: {state.route}")
   except Exception as e:
       print(f"Error during routing LLM call: {e}")
       state.route = "unclear" # Default to unclear on error
       print(f"Error occurred, defaulting route to: {state.route}")

   return state # Return the updated state


In [6]:
# Node 2: Execute Tool A
# This node simulates the action of using Tool A.
def execute_tool_a(state: GraphState) -> GraphState:
   """
   Simulates executing Tool A and updates the state with its output.
   """
   print("\n---EXECUTE TOOL A---")
   question = state.question
   # In a real application, this would contain the actual code to call an external tool/API
   # For simulation, we just generate a string based on the input question.
   tool_output = f"Tool A processed the question: '{question}'. Result: Simulated data processing output from Tool A."
   print(f"Simulated Tool A Output: {tool_output}")
   state.tool_a_output = tool_output # Store the tool's output in the state
   return state # Return the updated state

In [7]:
# Node 3: Execute Tool B
# This node simulates the action of using Tool B.
def execute_tool_b(state: GraphState) -> GraphState:
   """
   Simulates executing Tool B and updates the state with its output.
   """
   print("\n---EXECUTE TOOL B---")
   question = state.question
   # In a real application, this would contain the actual code to call a different external tool/API
   # For simulation, we generate a different string based on the input question.
   tool_output = f"Tool B processed the question: '{question}'. Result: Simulated information retrieval output from Tool B."
   print(f"Simulated Tool B Output: {tool_output}")
   state.tool_b_output = tool_output # Store the tool's output in the state
   return state # Return the updated state


In [8]:
# Node 4: Handle Unclear Questions
# This node provides a specific response when the routing is unclear.
def handle_unclear(state: GraphState) -> GraphState:
   """
   Generates a response for unclear questions and sets the final answer.
   """
   print("\n---HANDLE UNCLEAR---")
   question = state.question
   response = f"I'm sorry, I couldn't determine how to process your question: '{question}'. Could you please rephrase it? I can help with data processing (Tool A) or information retrieval (Tool B)."
   print(f"Unclear Response Generated: {response}")
   state.final_answer = response # Set the final answer in the state
   return state # Return the updated state

In [9]:
# Node 5: Synthesize Final Answer (after tool execution)
# This node takes the output from a tool and formats the final response for the user.
def synthesize_answer(state: GraphState) -> GraphState:
   """
   Synthesizes the final answer based on tool outputs stored in the state.
   """
   print("\n---SYNTHESIZE ANSWER---")
   question = state.question
   # Check which tool output is available in the state
   if state.tool_a_output:
       final_answer = f"Regarding your question: '{question}', here is the result from Tool A: {state.tool_a_output}"
   elif state.tool_b_output:
        final_answer = f"Regarding your question: '{question}', here is the result from Tool B: {state.tool_b_output}"
   else:
       # This case should ideally not be reached if routing works correctly and leads to a tool,
       # but included as a fallback for robustness.
       print("Warning: Synthesize node reached without tool output.")
       final_answer = f"Could not fully process your question: '{question}'."


   print(f"Synthesized Final Answer: {final_answer}")
   state.final_answer = final_answer # Set the final answer in the state
   return state # Return the updated state

## --- Define the Conditional Edge (Routing Logic) ---

In [10]:
# This function is called by LangGraph after the 'route_question' node executes.
# It determines the next node based on the 'state.route' value.
def route_decision_edge(state: GraphState) -> Literal["tool_a", "tool_b", "unclear", "synthesize"]:
   """
   Conditional edge that routes based on the 'route' state variable.
   Returns the name of the next node to transition to.
   """
   print(f"\n---ROUTING EDGE DECISION BASED ON STATE.ROUTE: '{state.route}'---")
   # LangGraph will use this return value to follow the corresponding edge.
   if state.route == "tool_a":
       return "tool_a" # Go to the execute_tool_a node
   elif state.route == "tool_b":
       return "tool_b" # Go to the execute_tool_b node
   elif state.route == "unclear":
       return "unclear" # Go to the handle_unclear node
   else:
       # Default or fallback if route value is unexpected (should be caught by Literal type hint, but good practice)
       print(f"Warning: Unexpected route decision '{state.route}', defaulting to unclear.")
       return "unclear"

In [11]:
# --- Build the LangGraph ---
if llm: # Only build graph if LLM initialized successfully
   workflow = StateGraph(GraphState) # Initialize the graph with the state definition


   # Add nodes to the graph workflow
   workflow.add_node("route_question", route_question)
   workflow.add_node("execute_tool_a", execute_tool_a)
   workflow.add_node("execute_tool_b", execute_tool_b)
   workflow.add_node("handle_unclear", handle_unclear)
   workflow.add_node("synthesize_answer", synthesize_answer)


   # Set the entry point - where the graph execution begins
   workflow.set_entry_point("route_question")


   # Add edges to define the transitions between nodes


   # The 'route_question' node's output determines the next step via a conditional edge
   workflow.add_conditional_edges(
       "route_question", # Source node: Execution comes from 'route_question'
       route_decision_edge, # The function that decides the next node based on state
       {
           "tool_a": "execute_tool_a", # If route_decision_edge returns "tool_a", transition to 'execute_tool_a' node
           "tool_b": "execute_tool_b", # If route_decision_edge returns "tool_b", transition to 'execute_tool_b' node
           "unclear": "handle_unclear", # If route_decision_edge returns "unclear", transition to 'handle_unclear' node
       }
   )


   # After executing Tool A or Tool B, the workflow goes to synthesize the answer (unconditional edges)
   workflow.add_edge("execute_tool_a", "synthesize_answer")
   workflow.add_edge("execute_tool_b", "synthesize_answer")


   # The 'handle_unclear' node is a terminal node - the workflow ends here
   workflow.add_edge("handle_unclear", END)


   # The 'synthesize_answer' node is also a terminal node - the workflow ends here
   workflow.add_edge("synthesize_answer", END)




   # Compile the graph into a runnable application
   app = workflow.compile()


   # --- Run the Graph with different questions ---
   # We will test the routing with questions intended for different tools and an unclear question.

   print("\n--- Running with a question intended for Tool A (Data Processing) ---")
   inputs_a = {"question": "Can you help me with some data processing?"} # Example question for Tool A
   output_a = app.invoke(inputs_a) # Invoke the graph with the input
   print(f"Final Output A: {output_a['final_answer']}") # Print the final answer from the state


   print("\n--- Running with an Unclear question ---")
   inputs_c = {"question": "Tell me a story about a dragon."} # Example unclear question
   output_c = app.invoke(inputs_c) # Invoke the graph with the input
   print(f"Final Output C: {output_c['final_answer']}") # Print the final answer from the state


   print("\n--- Running with Another Unclear question ---")
   inputs_d = {"question": "What's the weather like tomorrow?"} # Another example unclear question
   output_d = app.invoke(inputs_d) # Invoke the graph with the input
   print(f"Final Output D: {output_d['final_answer']}") # Print the final answer from the state


else:
   print("\nSkipping LangGraph execution due to LLM initialization failure.")


--- Running with a question intended for Tool A (Data Processing) ---

---ROUTE QUESTION---
LLM Routing decision raw output: 'tool_a'
State updated with route: tool_a

---ROUTING EDGE DECISION BASED ON STATE.ROUTE: 'tool_a'---

---EXECUTE TOOL A---
Simulated Tool A Output: Tool A processed the question: 'Can you help me with some data processing?'. Result: Simulated data processing output from Tool A.

---SYNTHESIZE ANSWER---
Synthesized Final Answer: Regarding your question: 'Can you help me with some data processing?', here is the result from Tool A: Tool A processed the question: 'Can you help me with some data processing?'. Result: Simulated data processing output from Tool A.
Final Output A: Regarding your question: 'Can you help me with some data processing?', here is the result from Tool A: Tool A processed the question: 'Can you help me with some data processing?'. Result: Simulated data processing output from Tool A.

--- Running with an Unclear question ---

---ROUTE QUESTIO